In [ ]:
import os, sys
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import hpc05
import holoviews as hv

import numpy as np
import kwant
import scipy.constants
import cmath
import functools as ft


import sns_system, spectrum, topology, supercurrent

from IPython.core.magic import register_cell_magic
import ipywidgets as widgets
run_cluster = widgets.Checkbox(
    value=False,
    description='Check to run cluster',
    disabled=False
)

interactive_shell = get_ipython()
@register_cell_magic
def cluster_mark_checked(line, cell):
    if run_cluster.value is True:
        run_cluster.value = False
        interactive_shell.run_cell(cell)
    else:
        return "Command not run. Check the box above to run."

del(cluster_mark_checked)
run_cluster

In [ ]:
%%cluster_mark_checked
# client, dview, lview = hpc05.connect_ipcluster(20, folder='two_dim_majoranas')
hpc05.kill_remote_ipcluster()
# client, dview, lview = hpc05.start_remote_and_connect(20, folder='two_dim_majoranas')

In [ ]:
client[:].use_pickle()

In [ ]:
with client[:].sync_imports():
    import numpy as np
    import scipy.constants
    import cmath

    import functools as ft

    import sns_system, spectrum, topology, supercurrent

In [ ]:
%%px
constants = dict(
    m_eff=0.026 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

# Make T shaped system

In [ ]:
%%px
def t_shaped_system(a, L_m, L_x, W_sc, L_lead, W_lead, offset_lead, W_t_lead,
                    transverse_soi=True,
                    mu_from_bottom_of_spin_orbit_bands=True,
                    k_x_in_sc=False, leaded=True, **_):

    #     HAMILTONIAN DEFINITIONS
    template_strings = sns_system.get_template_strings(
        transverse_soi, mu_from_bottom_of_spin_orbit_bands, k_x_in_sc)
    
    conservation_matrix = -supercurrent.sigz
    
    # TURN HAMILTONIAN STRINGS INTO TEMPLATES
    kwargs = dict(coords=('x', 'y'), grid_spacing=a)
    template_barrier = kwant.continuum.discretize(
        template_strings['ham_barrier'], **kwargs)
    template_normal = kwant.continuum.discretize(
        template_strings['ham_normal'], **kwargs)
    template_sc_left = kwant.continuum.discretize(
        template_strings['ham_sc_left'], **kwargs)
    template_sc_right = kwant.continuum.discretize(
        template_strings['ham_sc_right'], **kwargs)

    # SHAPE FUNCTIONS
    def shape_barrier(site):
        (x, y) = site.pos
        return (0 <= x < L_x) and (y == 0 or y == L_m)

    def shape_normal(site):
        (x, y) = site.pos
        return 0 <= x < L_x and a <= y < L_m

    def shape_left_sc(site):
        (x, y) = site.pos
        return 0 <= x < L_x and -W_sc - a < y < 0

    def shape_right_sc(site):
        (x, y) = site.pos
        return 0 <= x < L_x and L_m < y < L_m + a + W_sc
    
    syst = kwant.Builder(conservation_law=conservation_matrix)
    
    if leaded:
        lead_down = kwant.Builder(kwant.TranslationalSymmetry([0, -a]))
        lead_up = kwant.Builder(kwant.TranslationalSymmetry([0, a]))
        
        def shape_left_sc_lead(site):
            (x, y) = site.pos
            return offset_lead <= x <= offset_lead + W_lead #and -L_lead-W_sc - a < y < -W_sc

        def shape_right_sc_lead(site):
            (x, y) = site.pos
            return offset_lead <= x <= offset_lead + W_lead #and L_m + W_sc < y < L_m + a + W_sc + L_lead
    else:
        lead_down = syst
        lead_up = syst
        def shape_left_sc_lead(site):
            (x, y) = site.pos
            return offset_lead <= x <= offset_lead + W_lead and -L_lead-W_sc - a < y < -W_sc

        def shape_right_sc_lead(site):
            (x, y) = site.pos
            return offset_lead <= x <= offset_lead + W_lead and L_m + W_sc < y < L_m + a + W_sc + L_lead

    # BUILD FINITE SYSTEM
    lead_tunneling = kwant.Builder(kwant.TranslationalSymmetry([a, 0]), conservation_law=conservation_matrix)
    def shape_tunneling_lead(site):
        (x, y) = site.pos
        return L_m//2 - W_t_lead//2 <= y <= L_m//2 + W_t_lead//2
    lead_tunneling.fill(template_normal, shape_tunneling_lead, (0, L_m//2))

    syst.fill(template_normal, shape_normal, (a, 0)[::-1])
    syst.fill(template_barrier, shape_barrier, (0, 0)[::-1])
    syst.fill(template_barrier, shape_barrier, (L_m, 0)[::-1])
    syst.attach_lead(lead_tunneling)
    syst.attach_lead(lead_tunneling.reversed())
    if k_x_in_sc:
        syst.fill(template_sc_left, shape_left_sc, (0, -a))
        syst.fill(template_sc_right, shape_right_sc, (0, L_m + a))
        lead_up.fill(template_sc_right, shape_right_sc_lead, (offset_lead, L_m + a + W_sc))
        lead_down.fill(template_sc_left, shape_left_sc_lead, (offset_lead, -a + -W_sc))
        if leaded:
            syst.attach_lead(lead_down)
            syst.attach_lead(lead_up)

    else:
        for y in np.arange(0, L_x, a):
            syst.fill(template_sc_left, shape_left_sc, (y, -a))
    syst = syst.finalized()

    return syst

In [ ]:
%%px
syst_pars = dict(a=2.5, L_m=80, L_x=1600, W_sc=160, L_lead=800, W_lead=200, W_t_lead=20,
                 offset_lead=100, k_x_in_sc=True, leaded=False)
syst = t_shaped_system(**syst_pars)

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 74,
                 alpha_middle = 10,
                 alpha_left = 0,
                 alpha_right = 0,
                 Delta_left = .15,
                 Delta_right = .15,
                 B = 4,
                 phase = np.pi/2,
                 T = 0.0,
                 V = 0.0)

params = dict(**constants,
              **params_raw)

In [ ]:
import plotting_results
plotting_results.plot_syst(params=params, syst=syst);

In [ ]:
import formulas as f
vf = f.fermi_velocity(params['mu'], 0.026)
Eth = f.thouless_energy(vf, syst_pars['L_m'])
Eth/(params['g_factor_middle']*params['mu_B'])

In [ ]:
Eth

In [ ]:
kf = f.fermi_wavenumber(params['mu'], 0.026)
1/kf

In [ ]:
params['hbar']*vf/(np.pi*params['Delta_left'])

In [ ]:
f.density_to_mu(8e11/1e14, 0.026)

In [ ]:
params['hbar']**2 * np.pi**2 / (2 * 0.026/0.023*params['m_eff'] * (syst_pars['L_m'])**2)

In [ ]:
ham = syst.hamiltonian_submatrix(params=params, sparse=True)
d = kwant.operator.Density(syst)
e, ev = spectrum.mumps_eigsh(ham,4,0)

In [ ]:
e

In [ ]:
kwant.plotter.map(syst, np.abs(d(ev[:,0])), oversampling=1, fig_size=5); #2.5nm

In [ ]:
from multiprocessing import Pool

In [ ]:
p=Pool(12)

In [ ]:
def G(e):
    smat = kwant.smatrix(syst, energy=e, params=params)
    return smat.conductance_matrix()[0][0]

In [ ]:
G(.11)

In [ ]:
res = p.map(G, np.linspace(-.2, .2, 101))

In [ ]:
res

In [ ]:
hv.notebook_extension()
hv.Curve(res)